In [1]:

import sys
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.append(str(parent_dir))

import numpy as np
from tqdm import tqdm
import torch

from datasets import load_dataset
from src.dataset import Dataset
from src.model_dump import Dump
from src.model_phase import PhraseModel
from src.model_query import QueryModel

raw_datasets = load_dataset("squad")

In [ ]:
dataset = Dataset(model_checkpoint='SpanBERT/spanbert-base-cased')
dataset.load_ds(raw_datasets, size=500)

100%|██████████| 500/500 [00:00<00:00, 1540.99it/s]


In [ ]:
# lens = []
# for i in range(len(dataset.contexts)):
#     context_ids = dataset.tokenizer(dataset.contexts[i])['input_ids'][1:-1]
#     answer_ids = dataset.tokenizer(dataset.answers[i][0])['input_ids'][1:-1]
#     # print(dataset.contexts[i])
#     # print(dataset.answers[i][0])
#     # print(context_ids)
#     # print(answer_ids)
#     # start = context_ids.index(answer_ids[0]) 
#     # end = context_ids.index(answer_ids[-1])
    
#     start = dataset.spans_input_ids[i]['start']
#     end = dataset.spans_input_ids[i]['end']
#     print(len(context_ids), start, end)
#     print(dataset.tokenizer.decode(context_ids[start:end]), end='\n\n')

In [ ]:
   
phrasemodel = PhraseModel(hidden_dim=768, ds=dataset, model_checkpoint='SpanBERT/spanbert-base-cased')
optimizer = torch.optim.Adam(phrasemodel.parameters(), lr=0.01)
L = len(dataset.contexts)
indices = list(range(L))
np.random.shuffle(indices)
batch_size = 5
inputs_batched = [indices[i*batch_size:(i+1)*batch_size]
                          for i in range(L //batch_size+1)]

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
num_epochs = 10

scaler = torch.cuda.amp.GradScaler()


for epoch in range(num_epochs):
    # Forward pass
    losses = []
    for inputs in tqdm(inputs_batched):
    
        with torch.cuda.amp.autocast():
            loss = phrasemodel(inputs)

        # # Backward and optimize
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        losses.append(loss.item())
        
    print(np.mean(losses))

  0%|          | 0/101 [00:00<?, ?it/s]

In [5]:
dump = Dump(dataset, phrasemodel.model, 
              phrasemodel.model_1, phrasemodel.model_2,
              phrasemodel.tokenizer, hidden_dim=768)

dump.create_dump()

In [23]:
for i in range(100):
    output = dump.predict(i, k=100, verbose=True)
    if output[0]:
        print(output)

Q: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
C: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
('luck', 20.04961)
Q: What is in front of the Notre Dame Main Building?
C: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of

KeyboardInterrupt: 